In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np

In [3]:
# Parametros
num_epochs = 10
num_classes = 10
batch_size = 100
learning_rate = 0.001

DATA_PATH = 'C:\\Users\Hector\Documents\EOI - Artificial Intelligence and Deep Learning\Modulo 10 - Vision Artif con Google'
MODEL_STORE_PATH = 'C:\\Users\Hector\Documents\EOI - Artificial Intelligence and Deep Learning\Modulo 10 - Vision Artif con Google'

# Transformar el codigo y normalizarlo
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# Cargando el MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

9920512it [00:04, 2162945.65it/s]                                                                                      


Extracting C:\Users\Hector\Documents\EOI - Artificial Intelligence and Deep Learning\Modulo 10 - Vision Artif con Google\MNIST\raw\train-images-idx3-ubyte.gz


32768it [00:00, 106390.16it/s]                                                                                         


Extracting C:\Users\Hector\Documents\EOI - Artificial Intelligence and Deep Learning\Modulo 10 - Vision Artif con Google\MNIST\raw\train-labels-idx1-ubyte.gz


1654784it [00:01, 1120340.28it/s]                                                                                      


Extracting C:\Users\Hector\Documents\EOI - Artificial Intelligence and Deep Learning\Modulo 10 - Vision Artif con Google\MNIST\raw\t10k-images-idx3-ubyte.gz


8192it [00:00, 36132.85it/s]                                                                                           


Extracting C:\Users\Hector\Documents\EOI - Artificial Intelligence and Deep Learning\Modulo 10 - Vision Artif con Google\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [4]:
# Convolutional Network
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


model = ConvNet()

# Loss y optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Entrenando el modelo
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

# Probando el modelo
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
correct += (predicted == labels).sum().item()

Epoch [1/6], Step [100/600], Loss: 0.2662, Accuracy: 92.00%
Epoch [1/6], Step [200/600], Loss: 0.1781, Accuracy: 94.00%
Epoch [1/6], Step [300/600], Loss: 0.0644, Accuracy: 99.00%
Epoch [1/6], Step [400/600], Loss: 0.1403, Accuracy: 95.00%
Epoch [1/6], Step [500/600], Loss: 0.0760, Accuracy: 97.00%
Epoch [1/6], Step [600/600], Loss: 0.0707, Accuracy: 97.00%
Epoch [2/6], Step [100/600], Loss: 0.0584, Accuracy: 98.00%
Epoch [2/6], Step [200/600], Loss: 0.0543, Accuracy: 99.00%
Epoch [2/6], Step [300/600], Loss: 0.0686, Accuracy: 97.00%
Epoch [2/6], Step [400/600], Loss: 0.1082, Accuracy: 97.00%
Epoch [2/6], Step [500/600], Loss: 0.0680, Accuracy: 96.00%
Epoch [2/6], Step [600/600], Loss: 0.1656, Accuracy: 96.00%
Epoch [3/6], Step [100/600], Loss: 0.2113, Accuracy: 96.00%
Epoch [3/6], Step [200/600], Loss: 0.0681, Accuracy: 98.00%
Epoch [3/6], Step [300/600], Loss: 0.0633, Accuracy: 99.00%
Epoch [3/6], Step [400/600], Loss: 0.0371, Accuracy: 98.00%
Epoch [3/6], Step [500/600], Loss: 0.022

In [7]:
# Impresion de los resultados
print('Prueba de precision en el modelo que contiene 10000 imagenes de prueba: {} % (aprox.)'.format((correct / total) * 10000))


# Guardando el modelo
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')


# Plot
p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='PyTorch ConvNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(loss_list)), loss_list)
p.line(np.arange(len(loss_list)), np.array(acc_list) * 100, y_range_name='Accuracy', color='red')
show(p)

Prueba de precision en el modelo que contiene 10000 imagenes de prueba: 100.0 % (aprox.)
